In [2]:
!pip install boruta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns = data.feature_names)
y = pd.Series(data.target, name='target')

In [5]:
print("Dataset Shape:", X.shape)

Dataset Shape: (569, 30)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Feature Selection with Boruta
rf = RandomForestClassifier(n_estimators = 100, random_state = 42, n_jobs = -1)
boruta_selector = BorutaPy(estimator = rf, n_estimators = 'auto', random_state = 42)
boruta_selector.fit(X_train.values, y_train.values)

boruta_support = boruta_selector.support_
boruta_features = X_train.columns[boruta_support].tolist()
print("Selected Features by Boruta:", boruta_features)

Selected Features by Boruta: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean compactness', 'mean concavity', 'mean concave points', 'radius error', 'perimeter error', 'area error', 'concavity error', 'concave points error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


In [14]:
# Recursive Feature Elimination (RFE)
logreg = LogisticRegression(max_iter = 10000, solver = 'liblinear')
rfe_selector = RFE(estimator = logreg, n_features_to_select = 10, step = 1)
rfe_selector.fit(X_train, y_train)

rfe_support = rfe_selector.support_
rfe_features = X_train.columns[rfe_support].tolist()
print("Selected Features by RFE:", rfe_features)

Selected Features by RFE: ['mean radius', 'mean concavity', 'mean concave points', 'radius error', 'perimeter error', 'worst radius', 'worst perimeter', 'worst compactness', 'worst concavity', 'worst concave points']


In [15]:
# PCA
pca = PCA(n_components = 5)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print("Explained Variance by PCA Components:", pca.explained_variance_ratio_)

Explained Variance by PCA Components: [9.81351172e-01 1.65201603e-02 1.91058893e-03 1.25398750e-04
 8.11330525e-05]


In [16]:
def evaluate_model(X_train_sel, X_test_sel, y_train, y_test, name):
    clf = RandomForestClassifier(random_state = 42)
    clf.fit(X_train_sel, y_train)
    y_pred = clf.predict(X_test_sel)
    print(f"\n Performance using {name}:")
    print(classification_report(y_test, y_pred))

# Boruta
evaluate_model(X_train[boruta_features], X_test[boruta_features], y_train, y_test, "Boruta")

# RFE
evaluate_model(X_train[rfe_features], X_test[rfe_features], y_train, y_test, "RFE")

# PCA
evaluate_model(X_train_pca, X_test_pca, y_train, y_test, "PCA")


 Performance using Boruta:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        63
           1       0.96      0.99      0.98       108

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171


 Performance using RFE:
              precision    recall  f1-score   support

           0       0.95      0.90      0.93        63
           1       0.95      0.97      0.96       108

    accuracy                           0.95       171
   macro avg       0.95      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171


 Performance using PCA:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95        63
           1       0.96      0.98      0.97       108

    accuracy                           0.96       171
   macro avg       0.97      0.96      0.96       